In [1]:
import os
import pandas as pd
import numpy as np
import climate_econometrics_toolkit.user_api as api
from countrycode import countrycode as cc

/home/hayden-freedman/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
/home/hayden-freedman/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
cet_home = os.getenv("CETHOME")
reproduction_dir = "../../hierarchical_bayesian_drought_study_code/"

# Step 1: Construct dataset

In [49]:
tfp_data = pd.read_csv(f"{reproduction_dir}/data/TFP/AgTFPInternational2021_AG_TFP.csv", header=2)
# use former Sudan as Sudan
tfp_data = tfp_data.loc[tfp_data["Country/territory"] != "Sudan"]
natural_disasters_data = pd.read_csv(f"{reproduction_dir}/data/natural_disasters/emdat_1960-2024.csv")
countries_with_natural_disaster_data = set(natural_disasters_data.ISO)

In [50]:
# pre-process natural disasters data
nd_data = {}
for row in natural_disasters_data.iterrows():
    row = row[1]
    if row["ISO"] not in nd_data:
        nd_data[row["ISO"]] = []
    if row["Disaster Type"] == "Drought":
        if row["Start Year"] == row["End Year"]:
            nd_data[row["ISO"]].append(row["Start Year"])
        else:
            for year in range(row["Start Year"], row["End Year"]):
                nd_data[row["ISO"]].append(int(year))

In [51]:
ag_raster = f"{reproduction_dir}/data/CroplandPastureArea2000_Geotiff/Cropland2000_5m_resampled.tif"
shape_file = f"{reproduction_dir}/data/country_shapes/country.shp"

temp_raster = f"{reproduction_dir}/data/temp/monthly/shifted/air.2m.mon.mean.shifted.nc"
extracted_temp_data = api.extract_raster_data(temp_raster, shape_file, ag_raster)
temp_data = api.aggregate_raster_data(extracted_temp_data, shape_file, "temp", "mean", "FIPS_CNTRY", 12)
temp_data["country"] = cc(temp_data["FIPS_CNTRY"], origin="fips", destination="iso3c")
# update time column to reflect real year
temp_data["time"] = temp_data["time"] + 1948

precip_raster = f"{reproduction_dir}/data/precip/monthly/shifted/prate.mon.mean.shifted.nc"
extracted_precip_data = api.extract_raster_data(precip_raster, shape_file, ag_raster)
precip_data = api.aggregate_raster_data_to_year_level(extracted_precip_data, shape_file, "precip", "sum", "FIPS_CNTRY", 12)
precip_data["country"] = cc(precip_data["FIPS_CNTRY"], origin="fips", destination="iso3c")
# update time column to reflect real year
precip_data["time"] = precip_data["time"] + 1948

Warning 1: Latitude grid not spaced evenly.  Setting projection for grid spacing is within 0.1 degrees threshold.
/home/hayden-freedman/.local/lib/python3.10/site-packages/climate_econometrics_toolkit/prediction.py:46: RuntimeWarning: Mean of empty slice
  data.append([geo, period, np.nanmean(agg_mean)])
Warning 1: Latitude grid not spaced evenly.  Setting projection for grid spacing is within 0.1 degrees threshold.


In [52]:
reg_data = {"year":[],"iso3":[],"tfp":[],"temp":[],"precip":[],"drought":[]}
for country in set(temp_data["country"]):
    if country is not None and country in countries_with_natural_disaster_data:
        for year in range(1961,2022):
            reg_data["year"].append(year)
            reg_data["iso3"].append(country)
            try:
                reg_data["tfp"].append(tfp_data.loc[tfp_data["ISO3"]==country][str(year)].item())
            except ValueError:
                reg_data["tfp"].append(np.NaN)
            # celsius to kelvin
            reg_data["temp"].append(temp_data.loc[(temp_data.time == year) & (temp_data.country == country)]["temp"].item()-273.15)
            # precipitation rate per second to total monthly precipitation (X by approx. # of seconds in a month)
            reg_data["precip"].append(precip_data.loc[(precip_data.time == year) & (precip_data.country == country)]["precip"].item()*2.628e+6)
            reg_data["drought"].append(1 if year in nd_data[country] else 0)

In [54]:
pd.DataFrame.from_dict(reg_data).sort_values(["iso3","year"]).to_csv(f"{reproduction_dir}/data/regression/CET_tfp_regression_dataset.csv")

# Step 2: Build Model

In [81]:
api.load_dataset_from_file(f"{reproduction_dir}/data/regression/CET_tfp_regression_dataset.csv")
api.set_target_variable("tfp")
api.set_time_column("year")
api.set_panel_column("iso3")
api.add_covariates("temp")

api.add_transformation("tfp", ["ln", "fd"])
api.add_transformation("temp", "sq")

api.add_fixed_effects("year")
api.add_random_effect("drought", "iso3")

model_id = api.evaluate_model()

api.view_current_model()

/home/hayden-freedman/.local/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/home/hayden-freedman/.local/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/home/hayden-freedman/.local/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/home/hayden-freedman/.local/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/home/hayden-freedman/.local/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.

Intercept      -0.002554
temp            0.000857
sq_temp_       -0.000032
fe_1963_year    0.001085
fe_1964_year    0.011315
fe_1965_year   -0.002404
fe_1966_year    0.000100
fe_1967_year    0.014056
fe_1968_year   -0.003547
fe_1969_year   -0.004339
fe_1970_year    0.005468
fe_1971_year    0.005136
fe_1972_year   -0.019915
fe_1973_year   -0.002804
fe_1974_year    0.025285
fe_1975_year   -0.008112
fe_1976_year   -0.005341
fe_1977_year   -0.000296
fe_1978_year    0.016603
fe_1979_year   -0.006669
fe_1980_year    0.006148
fe_1981_year    0.006433
fe_1982_year    0.008756
fe_1983_year   -0.013095
fe_1984_year    0.013526
fe_1985_year    0.010280
fe_1986_year    0.010710
fe_1987_year   -0.006988
fe_1988_year    0.007459
fe_1989_year    0.019218
fe_1990_year    0.013513
fe_1991_year    0.001422
fe_1992_year   -0.004423
fe_1993_year    0.018012
fe_1994_year   -0.007239
fe_1995_year    0.023120
fe_1996_year    0.016863
fe_1997_year   -0.006764
fe_1998_year    0.006307
fe_1999_year    0.016625


In [82]:
api.run_bayesian_regression(model_id, 1000)

Fitting Bayesian model to dataset of length 9417 containing variables:  ['temp', 'sq(temp)', 'fe_1963_year', 'fe_1964_year', 'fe_1965_year', 'fe_1966_year', 'fe_1967_year', 'fe_1968_year', 'fe_1969_year', 'fe_1970_year', 'fe_1971_year', 'fe_1972_year', 'fe_1973_year', 'fe_1974_year', 'fe_1975_year', 'fe_1976_year', 'fe_1977_year', 'fe_1978_year', 'fe_1979_year', 'fe_1980_year', 'fe_1981_year', 'fe_1982_year', 'fe_1983_year', 'fe_1984_year', 'fe_1985_year', 'fe_1986_year', 'fe_1987_year', 'fe_1988_year', 'fe_1989_year', 'fe_1990_year', 'fe_1991_year', 'fe_1992_year', 'fe_1993_year', 'fe_1994_year', 'fe_1995_year', 'fe_1996_year', 'fe_1997_year', 'fe_1998_year', 'fe_1999_year', 'fe_2000_year', 'fe_2001_year', 'fe_2002_year', 'fe_2003_year', 'fe_2004_year', 'fe_2005_year', 'fe_2006_year', 'fe_2007_year', 'fe_2008_year', 'fe_2009_year', 'fe_2010_year', 'fe_2011_year', 'fe_2012_year', 'fe_2013_year', 'fe_2014_year', 'fe_2015_year', 'fe_2016_year', 'fe_2017_year', 'fe_2018_year', 'fe_2019_ye

Sampling: [covar_coefs, global_rs_mean, global_rs_sd, intercept, rs_coefs, rs_means, rs_sd, target_posterior, target_scale, target_std]
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [covar_coefs, intercept, global_rs_mean, global_rs_sd, rs_means, rs_sd, rs_coefs, target_scale, target_std]



KeyboardInterrupt



# Step 3: Compute Impacts

In [94]:
country_coefficients = pd.read_csv("../bayes_samples/coefficient_samples_1740668577.3381717.csv")

In [95]:
data = pd.read_csv(f"{reproduction_dir}/data/regression/CET_tfp_regression_dataset.csv")

In [96]:
res = api.call_user_prediction_function("multiply_geo_coefficients_by_data_column", ["iso3", 
    data, 
    country_coefficients[[col for col in country_coefficients.columns if col.startswith("drought_")]],
    "drought"])
percent_loss_by_country = api.call_user_prediction_function("convert_geo_log_loss_to_percent", [res])